In [ ]:
import os
from fastapi import FastAPI
from pydantic import BaseModel
from promptflow.core import Prompty
from promptflow.core import AzureOpenAIModelConfiguration
from dotenv import load_dotenv

if "AZURE_OPENAI_API_KEY" not in os.environ:
    # load environment variables from .env file
    load_dotenv('.env')

f = Prompty.load(source="chat.prompty")

# execute the flow as function
result = f(question="What is the capital of France?")  
result

"The capital of France is Paris! 🇫🇷✨ It's known for its rich history, art, fashion, and, of course, the iconic Eiffel Tower. If you ever get the chance to visit, it's a city full of wonders! 🗼❤️"

In [1]:
import json
import re
import os
from openai import AzureOpenAI
from pydub import AudioSegment
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound
from pytubefix import YouTube
from dotenv import load_dotenv

# Load environment variables
load_dotenv(".env")

# Initialize the OpenAI client
client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

# Utility functions for caching and sanitizing filenames (same as before)
def load_cache(cache_file):
    if os.path.exists(cache_file):
        with open(cache_file, 'r') as f:
            return json.load(f)
    else:
        return None

def save_cache(cache_file, data):
    with open(cache_file, 'w') as f:
        json.dump(data, f)

def sanitize_filename(filename):
    base_name, ext = os.path.splitext(filename)
    base_name = re.sub(r'[^\w\s-]', '', base_name)
    base_name = base_name.replace(' ', '_')
    base_name = base_name[:255]
    sanitized_filename = base_name + ext
    return sanitized_filename

def transcribe_audio(audio_path, client, chunk_size_ms=60000):  # Chunk size set to 1 minute
    audio = AudioSegment.from_file(audio_path)
    total_duration_ms = len(audio)
    transcription_text = ""
    
    # Process in chunks
    for start in range(0, total_duration_ms, chunk_size_ms):
        end = min(start + chunk_size_ms, total_duration_ms)
        chunk = audio[start:end]
        chunk_path = audio_path + f"_{start}_{end}.mp3"
        chunk.export(chunk_path, format="mp3")

        # Transcribe chunk
        with open(chunk_path, "rb") as f:
            response = client.audio.transcriptions.create(
                model="whisper",
                file=f,
            )
            transcription_text += response.text + " "
        os.remove(chunk_path)  # Clean up chunk file
    return transcription_text

def fetch_transcript(video_id):
    """
    Attempt to fetch the transcript using YouTubeTranscriptApi.
    If it fails, fall back to audio download and transcription.
    """
    try:
        # Try to fetch the transcript
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        # Combine all the transcript parts into one string
        full_text = " ".join([item['text'] for item in transcript])
        print("Transcript fetched successfully!")
        return full_text
    except TranscriptsDisabled:
        print("Transcripts are disabled for this video. Falling back to audio transcription.")
        return transcribe_audio_fallback(video_id)
    except NoTranscriptFound:
        print("No transcript found for this video. Falling back to audio transcription.")
        return transcribe_audio_fallback(video_id)
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

def download_audio(video_id, output_path="audio.mp4"):
    """
    Download the audio of the YouTube video using pytube.
    """
    try:
        video_url = f"https://www.youtube.com/watch?v={video_id}"
        yt = YouTube(video_url)
        stream = yt.streams.filter(only_audio=True).first()
        stream.download(filename=output_path)
        print(f"Audio downloaded successfully: {output_path}")
        return output_path
    except Exception as e:
        print(f"Failed to download audio: {e}")
        return None

def transcribe_audio_fallback(video_id):
    """
    Download audio and transcribe it using Whisper as a fallback.
    """
    audio_path = download_audio(video_id)
    if not audio_path:
        return "Failed to download audio for transcription."

    try:
        # Load the Whisper model
        result = transcribe_audio(audio_path, client)
        print("Audio transcription completed successfully!")
        return result["text"]
    except Exception as e:
        print(f"Failed to transcribe audio: {e}")
        return "Transcription failed."

In [3]:
video_id = "t73rFjUNY8823"  # Example video ID
transcript = fetch_transcript(video_id)
print("\nFinal Transcript:\n", transcript)

Transcript fetched successfully!

Final Transcript:
 [Music] all right welcome to another video so today we are going to be talking about promp flow and we're going to get you started with prompt flow and be able to deploy your own prompt flow inside of azure machine learning this is kind of what the ending is going to look like so you're actually going to have a flow that you can work with and manipulate you'll be able to chat with it interactively in the console uh you'll be able to deploy this to an online endpoint which in the end is going to look something like this to where we actually in aure machine learning will have an online endpoint with all the code that you have put together uh and deployed to a container instance uh kubernetes container uh that you have the ability to do auto scaling and even do monitoring of that if you so want to uh keep in mind that when this is deployed it not only it is just your code that makes the prompt flow work it does not include the open AI m

In [3]:
download_audio("t73rFjUNY88")

Audio downloaded successfully: audio.mp4


'audio.mp4'